Phase One

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving track database (1).csv to track database (1) (1).csv


** Database Before Normalization**

In [ ]:
import pandas as pd

# Read the uploaded CSV file using the filename from 'uploaded' dictionary
df = pd.read_csv("track database (1).csv")  # Use the filename directly

# Show the first few rows
df.head()

,track_ID,artist_name,genres,followers,artist_popularity,artist_url,track_name,album_name,release_date,duration_ms,explicit,track_popularity,energy,mode
0,0,Ariana Grande,pop,98934105,89,https://open.spotify.com/artist/66CXWjxzNUsdJx...,we can't be friends wait for your love,eternal sunshine,3/8/2024,228639,False,89,0.646,1
1,1,Ariana Grande,pop,98934105,85,https://open.spotify.com/artist/66CXWjxzNUsdJx...,the boy is mine,eternal sunshine,3/8/2024,173639,True,85,0.630,0
2,2,Ariana Grande,pop,98934105,83,https://open.spotify.com/artist/66CXWjxzNUsdJx...,intro,eternal sunshine,3/8/2024,92400,True,83,0.362,1
3,3,Ariana Grande,pop,98934105,80,https://open.spotify.com/artist/66CXWjxzNUsdJx...,Save Your Tears - Bonus Track,After Hours (Deluxe),3/20/2020,191013,False,80,0.825,1
4,4,Ariana Grande,pop,98934105,79,https://open.spotify.com/artist/66CXWjxzNUsdJx...,yes and?,eternal sunshine,3/8/2024,214994,True,79,0.775,1


In [ ]:
# Connect to SQLite and create tables
import sqlite3
import pandas as pd

conn = sqlite3.connect("university.db")
cursor = conn.cursor()

# Drop the tables if they exist to avoid the error
cursor.execute("DROP TABLE IF EXISTS Artist")
cursor.execute("DROP TABLE IF EXISTS Artist_Genres")
cursor.execute("DROP TABLE IF EXISTS Album")
cursor.execute("DROP TABLE IF EXISTS Track")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Artist (
    artist_name TEXT PRIMARY KEY,
    followers INTEGER,
    artist_popularity INTEGER,
    artist_url TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Artist_Genres (
    artist_name TEXT,
    genres TEXT,
    PRIMARY KEY (artist_name, genres),
    FOREIGN KEY (artist_name) REFERENCES Artist(artist_name)
)
""")


cursor.execute("""
CREATE TABLE Album (
    album_name TEXT PRIMARY KEY,
    artist_name TEXT,
    FOREIGN KEY (artist_name) REFERENCES Artist(artist_name)
);
""")

cursor.execute("""
CREATE TABLE Track (
    track_ID TEXT PRIMARY KEY,
    track_name TEXT,
    artist_name TEXT,
    album_name TEXT,
    release_date TEXT,
    duration_ms INTEGER,
    explicit BOOLEAN,
    track_popularity INTEGER,
    energy REAL,
    mode INTEGER,
    FOREIGN KEY (artist_name) REFERENCES Artist(artist_name),
    FOREIGN KEY (album_name) REFERENCES Album(album_name)
);
""")



conn.commit()
print("✅ Tables created")

✅ Tables created


In [ ]:
import pandas as pd
import sqlite3
import re


conn = sqlite3.connect("university.db")
cursor = conn.cursor()

# CSV
songs = pd.read_csv("track database (1).csv")

# spilt the gener multi-value into specific one value in each row
songs['genres'] = songs['genres'].astype(str).apply(lambda x: re.sub(r"[\[\]' ]", "", x))
songs['genres'] = songs['genres'].str.split(',')

# add each value in one row
artist_genres_df = songs.explode('genres')[['artist_name', 'genres']].drop_duplicates()
artist_genres_df.rename(columns={'genres': 'genre'}, inplace=True)

# شعبية له تعديل هنا: اختيار صف واحد لكل فنان بأعلى artist_popularity
# اختار المتوسط للشعبية او اعلى شعبية ؟؟؟؟
artist_df = songs.sort_values('artist_popularity', ascending=False).drop_duplicates(subset='artist_name')

# تحديد الأعمدة المطلوبة فقط
artist_df = artist_df[['artist_name', 'followers', 'artist_popularity', 'artist_url']]

# باقي الجداول
album_df = songs[['album_name', 'artist_name']].drop_duplicates()
track_df = songs[['track_ID', 'track_name', 'artist_name', 'album_name', 'release_date', 'duration_ms', 'explicit', 'track_popularity', 'energy', 'mode']].drop_duplicates()


artist_df.to_sql('Artist', conn, if_exists='replace', index=False)
artist_genres_df.to_sql('Artist_Genres', conn, if_exists='replace', index=False)
album_df.to_sql('Album', conn, if_exists='replace', index=False)
track_df.to_sql('Track', conn, if_exists='replace', index=False)

conn.commit()
print("✅ Data inserted into SQLite database with unique artists by highest popularity")


✅ Data inserted into SQLite database with unique artists by highest popularity


In [ ]:
# 📥 Load CSV and insert into tables
songs = pd.read_csv("track database (1).csv")

# Select only the relevant columns for each table before insertion:
artist_data = songs[['artist_name', 'followers', 'artist_popularity', 'artist_url']].drop_duplicates()
artist_data.to_sql("Artist", conn, if_exists="replace", index=False) # Changed to 'replace'

artist_genres_data = songs[['artist_name', 'genres']].drop_duplicates()
# Assuming 'genres' column exists in your CSV for Artist_Genres

# ... (Similarly select relevant data for Album and Track tables)

album_data = songs[['album_name', 'artist_name']].drop_duplicates()
album_data.to_sql("Album", conn, if_exists="replace", index=False) # Changed to 'replace'

# Fix: Replace 'track_id' with the actual column name from your CSV
# Print songs.columns to see all column names and find the correct one
print(songs.columns)
track_data = songs[['track_ID', 'track_name', 'artist_name', 'album_name', 'release_date', 'duration_ms', 'explicit', 'track_popularity', 'energy', 'mode']].drop_duplicates()
track_data.to_sql("Track", conn, if_exists="replace", index=False) # Changed to 'replace'




Index(['track_ID', 'artist_name', 'genres', 'followers', 'artist_popularity',
       'artist_url', 'track_name', 'album_name', 'release_date', 'duration_ms',
       'explicit', 'track_popularity', 'energy', 'mode'],
      dtype='object')
✅ Data inserted


In [ ]:
#  View tables After Normalization
print(" Artist_Genres After Normalization :")
for row in cursor.execute("SELECT * FROM Artist_Genres "):
    print(row)
#     df = pd.read_sql_query(query, conn)
# df

 Artist_Genres After Normalization :
('Ariana Grande', 'pop')
('Adele', 'britishsoul')
('A$AP Rocky', 'rap')
('A$AP Rocky', 'hiphop')
('Taylor Swift', 'pop')
('Arctic Monkeys', 'garagerock')
('Arctic Monkeys', 'modernrock')
('Arctic Monkeys', 'permanentwave')
('Arctic Monkeys', 'rock')
('Arctic Monkeys', 'sheffieldindie')
('Alice In Chains', 'alternativemetal')
('Alice In Chains', 'alternativerock')
('Alice In Chains', 'grunge')
('Alice In Chains', 'hardrock')
('Alice In Chains', 'numetal')
('Alice In Chains', 'rock')
('Anuel AA', 'reggaeton')
('Anuel AA', 'reggaetonflow')
('Anuel AA', 'trapboricua')
('Anuel AA', 'traplatino')
('Anuel AA', 'urbanolatino')
('ABBA', 'europop')
('ABBA', 'swedishpop')
('AJR', 'pov:indie')
('AC/DC', 'australianrock')
('AC/DC', 'hardrock')
('AC/DC', 'rock')
('Alex G', 'phillyindie')
('Alex G', 'pov:indie')
('Alex G', 'slackerrock')
('A Boogie Wit da Hoodie', 'melodicrap')
('A Boogie Wit da Hoodie', 'rap')
('A Boogie Wit da Hoodie', 'trap')
('Addison Rae', 's

In [ ]:
#  View tables After Normalization
print(" Track  After Normalization:")
for row in cursor.execute("SELECT * FROM Track "):
    print(row)

 Track  After Normalization:
(0, "we can't be friends wait for your love", 'Ariana Grande', 'eternal sunshine', '3/8/2024', 228639, 0, 89, 0.646, 1)
(1, 'the boy is mine', 'Ariana Grande', 'eternal sunshine', '3/8/2024', 173639, 1, 85, 0.63, 0)
(2, 'intro ', 'Ariana Grande', 'eternal sunshine', '3/8/2024', 92400, 1, 83, 0.362, 1)
(3, 'Save Your Tears   - Bonus Track', 'Ariana Grande', 'After Hours (Deluxe)', '3/20/2020', 191013, 0, 80, 0.825, 1)
(4, 'yes and?', 'Ariana Grande', 'eternal sunshine', '3/8/2024', 214994, 1, 79, 0.775, 1)
(5, 'One Last Time', 'Ariana Grande', 'My Everything - Deluxe', '8/22/2014', 197266, 0, 82, 0.593, 1)
(6, 'Die For You  - Remix', 'Ariana Grande', 'Starboy (Deluxe)', '3/14/2023', 232857, 0, 78, 0.485, 0)
(7, '7 rings', 'Ariana Grande', 'thank u, next', '2/8/2019', 178626, 1, 82, 0.317, 0)
(8, 'Dangerous Woman', 'Ariana Grande', 'Dangerous Woman', '5/20/2016', 235946, 0, 70, 0.602, 0)
(9, 'Into You', 'Ariana Grande', 'Dangerous Woman', '5/20/2016', 244453,

In [ ]:
# 📋 View tables After Normalization
print("🎓 Artist After Normalization:")
for row in cursor.execute("SELECT * FROM Artist "):
    print(row)

🎓 Artist After Normalization:
('Taylor Swift', 119286617, 92, 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02')
('Arctic Monkeys', 26055385, 91, 'https://open.spotify.com/artist/7Ln80lUS6He07XvHI8qqHH')
('Ariana Grande', 98934105, 89, 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR')
('Gracie Abrams', 2232980, 87, 'https://open.spotify.com/artist/4tuJ0bMpJh08umKkEXKUI5')
('AFROJACK', 3922609, 86, 'https://open.spotify.com/artist/4D75GcNG95ebPtNvoNVXhz')
('AC/DC', 29139022, 86, 'https://open.spotify.com/artist/711MCceyCBcFnzjGY4Q7Un')
('Ne-Yo', 9033803, 86, 'https://open.spotify.com/artist/21E3waRsmPlU7jZsS13rcj')
('Avicii', 22976158, 86, 'https://open.spotify.com/artist/1vCWHaC5f2uS3yhpwWbIA6')
('Gunna', 7819110, 86, 'https://open.spotify.com/artist/2hlmm7s2ICUX0LVIhVFlZQ')
('ABBA', 12698682, 85, 'https://open.spotify.com/artist/0LcJLqbBmaGUft1e9Mm8HV')
('Anne-Marie', 12038839, 85, 'https://open.spotify.com/artist/1zNqDE7qDGCsyzJwohVaoX')
('Ed Sheeran', 115449586, 85, '

In [ ]:
#  View tables After Normalization
print(" Album  After Normalization")
for row in cursor.execute("SELECT * FROM Album "):
    print(row)

 Album  After Normalization
('eternal sunshine', 'Ariana Grande')
('After Hours (Deluxe)', 'Ariana Grande')
('My Everything - Deluxe', 'Ariana Grande')
('Starboy (Deluxe)', 'Ariana Grande')
('thank u, next', 'Ariana Grande')
('Dangerous Woman', 'Ariana Grande')
('30', 'Adele')
('21', 'Adele')
('Skyfall', 'Adele')
('25', 'Adele')
('19', 'Adele')
('Sundress', 'A$AP Rocky')
('I Smoked Away My Brain ', 'A$AP Rocky')
('TESTING', 'A$AP Rocky')
('AT.LONG.LAST.A$AP', 'A$AP Rocky')
('HIGHJACK', 'A$AP Rocky')
('LONG.LIVE.A$AP (Deluxe Version)', 'A$AP Rocky')
('METRO BOOMIN PRESENTS SPIDER-MAN: ACROSS THE SPIDER-VERSE (SOUNDTRACK FROM AND INSPIRED BY THE MOTION PICTURE)', 'A$AP Rocky')
('Lover', 'Taylor Swift')
('THE TORTURED POETS DEPARTMENT', 'Taylor Swift')
('folklore', 'Taylor Swift')
('Midnights', 'Taylor Swift')
('reputation', 'Taylor Swift')
('AM', 'Arctic Monkeys')
('Favourite Worst Nightmare ', 'Arctic Monkeys')
('Favourite Worst Nightmare (Standard Version)', 'Arctic Monkeys')
("Whateve

**Insertion**

**Insert Data Into the artiest Table **

In [ ]:
# Insert into Artist table
cursor.execute("""
INSERT INTO Artist (artist_name, followers, artist_popularity, artist_url)
VALUES (?, ?, ?, ?)
""", (
    "The Weeknd", 82000000, 96, "https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ"
))


**Add Genre for The Weeknd**

In [ ]:
cursor.execute("""
INSERT INTO Artist_Genres (artist_name, genre)
VALUES (?, ?)
""", (
    "The Weeknd", "r&b"
))


**Insert to album**

In [ ]:
cursor.execute("""
INSERT INTO Album (album_name, artist_name)
VALUES (?, ?)
""", (
    "After Hours", "The Weeknd"
))


insert To track

In [ ]:
cursor.execute("""
INSERT INTO Track (
    track_ID, track_name, artist_name, album_name,
    release_date, duration_ms, explicit, track_popularity,
    energy, mode
)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (
    "TW-AH-01", "Blinding Lights", "The Weeknd", "After Hours",
    "2020-03-20", 200040, False, 94, 0.73, 1
))


Read The Data

In [ ]:
cursor.execute("SELECT * FROM Artist WHERE artist_name = ?", ("Ariana Grande",))
result = cursor.fetchone()
print(result)


('Ariana Grande', 98934105, 89, 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR')


UPDATE THE FOLLOWING'S ARTIST

In [ ]:
cursor.execute("""
UPDATE Artist
SET followers = ?
WHERE artist_name = ?
""", (992000000, "Ariana Grande"))
conn.commit()


In [ ]:
cursor.execute("SELECT * FROM Artist WHERE artist_name = ?", ("Ariana Grande",))
result = cursor.fetchone()  #print the
print(result)

('Ariana Grande', 992000000, 89, 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR')


**Delete the artist **

In [ ]:
conn = sqlite3.connect("university.db")
cursor = conn.cursor()

# Delete the track
cursor.execute("DELETE FROM Track WHERE track_name = ?", ("the boy is mine",))
conn.commit()

# Check if it still exists
cursor.execute("SELECT * FROM Track WHERE track_name = ?", ("the boy is mine",))
result = cursor.fetchall()

if result:
    print("Track still exists:", result)
else:
    print("Track successfully deleted.")

conn.close()


Track successfully deleted.


In [ ]:
import time
import sqlite3


conn = sqlite3.connect("university.db")
cursor = conn.cursor()

# Time measurement without indexing
start = time.time()
cursor.execute("SELECT * FROM Track WHERE artist_name = 'Andrew Underberg'")
cursor.fetchall()
print(" Time without index:", time.time() - start)

# create insex to artist_name
cursor.execute("CREATE INDEX IF NOT EXISTS idx_track_artist ON Track(artist_name)")
conn.commit()

# Time Measurement after indexing
start = time.time()
cursor.execute("SELECT * FROM Track WHERE artist_name = 'Andrew Underberg'")
cursor.fetchall()
print(" Time with index:", time.time() - start)

conn.close()


 Time without index: 0.0010018348693847656
 Time with index: 0.0003085136413574219


***Download The tables ***

In [ ]:
import pandas as pd
from google.colab import files

# ... (your existing code to connect to the database and fetch data) ...

# Fetch data from the Artist_Genres table
artist_genres_df = pd.read_sql_query("SELECT * FROM Artist_Genres", conn)

# Save the DataFrame to an Excel file
artist_genres_df.to_excel("Artist_Genres.xlsx", index=False)

# Now you can download the file
files.download("Artist_Genres.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>